# Building Agents with LangGraph & Llama Stack

In the last notebook, you built a ReAct agent **from scratch**. You wrote:
- Manual response parsing with regex
- Custom iteration loops
- State management
- Tool execution logic
- Error handling

That was ~200+ lines of code. Now let's see how **LangGraph** makes this dramatically simpler.

## What We'll Build

A knowledge-based assistant that:
1. Searches documents for answers (RAG)
2. Schedules meetings with professors if it can't answer

Same capabilities. Way less code.

## Setup: Install Dependencies

In [ ]:
!pip3 install -q langgraph==0.6.7 langchain-openai==0.3.32 langchain-core==0.3.75 llama-stack-client==0.3.0 langchain==0.3.27

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from llama_stack_client import LlamaStackClient
from langchain_mcp_adapters.client import MultiServerMCPClient
import json

In [ ]:
#Disable some logs
import logging

logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger().setLevel(logging.WARNING)

## Connect to Llama Stack

LangGraph connects to Llama Stack using the **Responses API**, which provides enhanced functionality including direct MCP tool binding.

**Important**: When using `use_responses_api=True`, the base URL should be:
- ✅ `http://llama-stack-service:8321/v1` (Responses API endpoint)
- ❌ NOT `http://llama-stack-service:8321/v1/openai/v1` (that's for standard Chat Completions)

The Responses API is what enables the seamless MCP integration!

In [ ]:
# LangGraph connects via Llama Stack Responses API
# When using use_responses_api=True, the base should be /v1 (not /v1/openai/v1)
# The Responses API supports tool binding for MCP integration
llm = ChatOpenAI(
    openai_api_base="http://llama-stack-service:8321/v1",
    model="llama32",
    openai_api_key="not-needed",
    use_responses_api=True,
    temperature=0.1
)

# Also connect standard client for RAG
llama_client = LlamaStackClient(base_url="http://llama-stack-service:8321")

print(f"✅ Connected to Llama Stack with Responses API enabled")

## Define Tools

We'll give the agent access to:
1. **RAG Search** - Search the knowledge base for information
2. **Professor Directory** - Look up available professors by expertise  
3. **MCP Calendar Tools** - Direct MCP server integration!

For the MCP calendar integration, we use Llama Stack's Responses API which supports **direct tool binding**. We simply include the MCP binding in the tools list, and LangGraph handles everything automatically:

- The MCP binding connects to the real calendar server you configured
- All 9 calendar tools become available (create_event, get_upcoming_events, search_events, etc.)
- Tool calls are handled asynchronously through Llama Stack
- No manual wrapper functions needed!

This is the same MCP calendar server you used in previous notebooks (`2-mcp-servers.ipynb` and `3-agentic-workflows.ipynb`).

In [ ]:
# First, let's get our vector store for RAG
try:
    vector_stores = llama_client.vector_stores.list()
    
    if vector_stores.data and len(vector_stores.data) > 0:
        vector_store = vector_stores.data[0]
        print(f"✅ Using vector store: {vector_store.id}")
    else:
        raise Exception("Can't find an existing vector store")

    # Delete and recreate the vector store so that it properly connects to the underlying vector db after restarting LLS pod
    llama_client.vector_stores.delete(vector_store_id=vector_store.id)
    vector_store = llama_client.vector_stores.create(
        name="my_citations_db",
        extra_body={
            "embedding_model": "all-MiniLM-L6-v2",
            "embedding_dimension": 384,
            "provider_id": "milvus",
            "vector_db_id": "test"
        }
    )

except Exception as e:
    print(f"⚠️  Vector store setup failed: {e}")
    vector_store = None

In [ ]:
# Professor directory - in a real system this would be a database
PROFESSORS = {
    "Dr. Sarah Chen": {
        "department": "Computer Science",
        "expertise": ["Machine Learning", "Neural Networks", "AI Ethics", "Agentic Workflows"],
        "email": "s.chen@university.edu"
    },
    "Prof. Michael Rodriguez": {
        "department": "Physics",
        "expertise": ["Quantum Mechanics", "Particle Physics", "Quantum Chromodynamics"],
        "email": "m.rodriguez@university.edu"
    },
    "Dr. Emily Thompson": {
        "department": "Biology",
        "expertise": ["Botany", "Ecology", "Forest Canopy Structure", "Plant Biology"],
        "email": "e.thompson@university.edu"
    },
    "Prof. James Wilson": {
        "department": "Computer Science",
        "expertise": ["Distributed Systems", "Cloud Computing", "Software Architecture"],
        "email": "j.wilson@university.edu"
    }
}


@tool
def search_knowledge_base(query: str) -> str:
    """Search through documents to find information. Use this when the user asks about concepts, definitions, or topics."""
    if not vector_store:
        return "Error: Knowledge base not available. Please set up the vector store first."
    
    try:
        results = llama_client.vector_stores.search(
            vector_store_id=vector_store.id,
            query=query,
            max_num_results=3,
            search_mode="vector"
        )
        
        if not results.data:
            return "No relevant information found in the knowledge base."
        
        formatted_results = []
        for i, result in enumerate(results.data, 1):
            content = result.content if hasattr(result, 'content') else str(result)
            formatted_results.append(f"Result {i}: {content}")
        
        return "\n\n".join(formatted_results)
    except Exception as e:
        return f"Error searching knowledge base: {str(e)}"


@tool
def find_professors_by_expertise(topic: str) -> str:
    """Find professors who have expertise in a specific topic or subject area.
    
    Args:
        topic: The topic or subject area to search for (e.g., 'Machine Learning', 'Quantum Physics', 'Botany')
    
    Returns:
        List of professors with matching expertise, including their name, department, and contact info
    """
    matching_profs = []
    
    for name, info in PROFESSORS.items():
        # Check if the topic matches any of their expertise areas
        if any(topic.lower() in exp.lower() or exp.lower() in topic.lower() for exp in info["expertise"]):
            matching_profs.append((name, info))
    
    if not matching_profs:
        # If no exact match, show all professors
        result = f"No professors found with specific expertise in '{topic}'.\n\n"
        result += "Available professors:\n\n"
        for name, info in PROFESSORS.items():
            result += f"**{name}** - {info['department']}\n"
            result += f"  Expertise: {', '.join(info['expertise'])}\n"
            result += f"  Email: {info['email']}\n\n"
        return result
    
    result = f"Professors with expertise in '{topic}':\n\n"
    for name, info in matching_profs:
        result += f"**{name}** - {info['department']}\n"
        result += f"  Expertise: {', '.join(info['expertise'])}\n"
        result += f"  Email: {info['email']}\n\n"
    
    return result


# Define all tools: custom tools + MCP binding
tools = [
    search_knowledge_base,
    find_professors_by_expertise,
    # MCP calendar server binding
    {
        "type": "mcp",
        "server_label": "canopy-calendar",
        "server_url": "http://canopy-mcp-calendar-mcp-server:8080/sse",
        "require_approval": "never",
    }
]

print("✅ Tools defined:")
print("   Custom tools:")
print("     - search_knowledge_base")
print("     - find_professors_by_expertise")
print("   MCP calendar server:")
print("     - canopy-calendar (provides 9 calendar tools via Responses API)")

## Build the Agent

This is where the magic happens. With LangGraph, creating a ReAct agent is **one line of code**.

No parsing. No loops. No state management. LangGraph handles it all.

**What's happening behind the scenes:**
- We pass the MCP calendar binding in the `tools` list
- `create_react_agent()` automatically binds all tools (custom + MCP) to the LLM
- The Responses API enables MCP tool binding, allowing direct integration with the MCP server
- LangGraph handles all the async MCP function calls automatically
- The agent can now interact with your actual calendar database!

In [ ]:
# Create the ReAct agent with all tools
agent = create_react_agent(
    llm,
    tools,
    checkpointer=MemorySaver(),
)

print("✅ ReAct agent created!")
print("\n🎯 The agent has access to:")
print("   • 2 custom tools (RAG search, professor directory)")
print("   • 9 MCP calendar tools (automatically bound via Responses API)")
print("\n🎯 Compare this to the 200+ lines you wrote before...")

We can see a small diagram of our agent as well

In [ ]:
agent

### What Just Happened?

That one function call replaces ALL of this from your previous notebook:
- ❌ `parse_react_response()` - Manual regex parsing
- ❌ `execute_tool()` - Tool routing logic
- ❌ `run_react_agent()` - The entire iteration loop
- ❌ Conversation history management
- ❌ Error handling and retry logic
- ❌ Stopping condition checks

LangGraph does **all of this automatically**.

## Test the Agent

Let's see it in action!

In [ ]:
import uuid
from textwrap import indent
from langchain_core.messages import HumanMessage
import pprint

def run_agent(question: str, thread_id: str | None = None):
    """Run the LangGraph agent, stream values, and show MCP tool calls + final answer."""
    thread_id = thread_id or str(uuid.uuid4())

    print("\n" + "="*80)
    print("USER INPUT")
    print("="*80 + "\n")
    print(question, "\n")

    config = {"configurable": {"thread_id": thread_id}}
    inputs = {"messages": [HumanMessage(content=question)]}

    seen_messages = 0
    final_state = None
    tool_header_printed = False

    # IMPORTANT: stream_mode="values"
    for state in agent.stream(inputs, config, stream_mode="values"):

        messages = state.get("messages", [])
        new_messages = messages[seen_messages:]
        seen_messages = len(messages)

        for msg in new_messages:
            msg_type = getattr(msg, "type", None)

            if msg_type == "ai":
                if getattr(msg, "tool_calls", None):
                    if not tool_header_printed:
                        print("="*80)
                        print("TOOL CALLS")
                        print("="*80 + "\n")
                        tool_header_printed = True

                    for tc in msg.tool_calls:
                        name = tc.get("name")
                        args = tc.get("args")
                        print(f"🔧 Calling tool: {name}")
                        print(f"   Args: {args}\n")
                else:
                    tool_outputs = msg.additional_kwargs.get("tool_outputs", [])

                    # MCP tool calls live here
                    for t in tool_outputs:
                        if t.get("type") == "mcp_call":
                            if not tool_header_printed:
                                print("="*80)
                                print("TOOL CALLS")
                                print("="*80 + "\n")
                                tool_header_printed = True

                            name = t.get("name")
                            server = t.get("server_label")
                            args = t.get("arguments")
                            error = t.get("error", "")
                            output = t.get("output", "")

                            print(f"🔧 MCP TOOL CALL: {name}  (server: {server})")
                            print(f"   args: {args}")
                            if output:
                                print("   output:")
                                print(indent(str(output), "      "))
                            elif error:
                                print("   error:")
                                print(indent(str(error), "      "))
                            print()

            elif msg_type == "tool":
                # You can customize how much of the result to show
                print(f"📦 Tool result ({msg.name}): {str(msg.content)[:200]}...\n")

        final_state = state

    if final_state is None:
        print("⚠️ Agent produced no state")
        return

    # Final assistant message = last AI message
    messages = final_state.get("messages", [])
    final_answer = None
    for msg in reversed(messages):
        if getattr(msg, "type", None) == "ai":
            final_answer = msg
            break

    print("="*80)
    print("FINAL AGENT RESPONSE")
    print("="*80 + "\n")
    if final_answer:
        print(final_answer.content[0]["text"])
    else:
        print("(No assistant answer found)")
    print("="*80 + "\n")


### Example 1: Knowledge Base Search

The agent should search the knowledge base for this information.

**Note**: This requires documents in your vector store. If you haven't done the RAG notebooks (`5-rag/2-intro-to-RAG.ipynb`), the knowledge base will be empty.

In [ ]:
run_agent("What is the structure of a forest canopy in botany?")

### Example 2: Finding an Expert & Scheduling

When the agent can't answer a question, watch it:
1. Search for professors with relevant expertise
2. Schedule a calendar meeting

In [ ]:
run_agent("I need help understanding quantum chromodynamics. Can you find me an expert and schedule a meeting for December 1st, 2025 at 2pm for one hour?")

### Example 3: Using the MCP Calendar

The agent can check your actual calendar and search for events!

In [ ]:
run_agent("What upcoming lectures do I have?")

## Try It Yourself!

Now experiment:
1. Ask questions that require knowledge base searches
2. Ask questions that should trigger meeting scheduling
3. Try to make the agent use both tools in sequence

See how the agent autonomously reasons about which tool(s) to use!

In [ ]:
# Your turn!
run_agent("YOUR QUESTION HERE")